In [1]:
import tensorflow as tf

print("TensorFlow version: ", tf.__version__)
print("Num GPUs Available: ", len(tf.config.list_physical_devices("GPU")))


device_name = tf.test.gpu_device_name()
if device_name:
    print(f"Device name: '{device_name}'")
    print(
        "Device properties:",
        tf.config.experimental.get_device_details(
            tf.config.experimental.list_physical_devices("GPU")[0]
        ),
    )
else:
    print("No GPU device found.")

# Check bfloat16 support
bf16_supported = any(
    "bfloat16"
    in tf.config.experimental.get_device_details(gpu_device).get(
        "compute_capability", ""
    )
    for gpu_device in tf.config.experimental.list_physical_devices("GPU")
)

print("Suporta bfloat16." if bf16_supported else "Não suporta bfloat16.")

TensorFlow version:  2.12.0
Num GPUs Available:  1
Device name: '/device:GPU:0'
Device properties: {'compute_capability': (7, 5), 'device_name': 'NVIDIA GeForce RTX 2060 SUPER'}
Não suporta bfloat16.


# Tokenizadores - Introdução

Na maioria das tarefas de PNL, o passo inicial na preparação dos seus dados é extrair um vocabulário de palavras do seu *corpus* (ou seja, textos de entrada). Você precisará definir como representar os textos em representações numéricas que podem ser usadas para treinar uma rede neural. Essas representações são chamadas de *tokens* e Tensorflow e Keras facilitam sua geração usando suas APIs. Você verá como fazer isso nas próximas células.

## Gerando o Vocabulário

Neste notebook, você verá primeiro como pode fornecer um dicionário de consulta para cada palavra. O código abaixo pega uma lista de frases, depois pega cada palavra dessas frases e a atribui a um número inteiro. Isso é feito usando o método [fit_on_texts()](https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/text/Tokenizer#fit_on_texts) e você pode obter o resultado olhando a propriedade `word_index`. <u>Palavras mais frequentes têm um índice menor</u>.

In [2]:
from tensorflow.keras.preprocessing.text import Tokenizer

# Define input sentences
sentences = ["i love my dog", "I, love my cat"]

# Initialize the Tokenizer class
tokenizer = Tokenizer(num_words=100)

# Generate indices for each word in the corpus
tokenizer.fit_on_texts(sentences)

# Get the indices and print it
word_index = tokenizer.word_index
print(word_index)

{'i': 1, 'love': 2, 'my': 3, 'dog': 4, 'cat': 5}


O parâmetro `num_words` usado no inicializador especifica o número máximo de palavras menos uma (com base na frequência) a serem mantidas ao gerar sequências. Você verá isso em um exercício posterior. Por enquanto, o importante é notar que isso não afeta como o dicionário `word_index` é gerado. Você pode tentar passar `1` em vez de `100`, como mostrado na próxima célula, e obterá o mesmo `word_index`.

Observe também que, por padrão, toda a pontuação é ignorada e as palavras são convertidas para minúsculas. Você pode substituir esses comportamentos modificando os argumentos `filters` e `lower` da classe `Tokenizer`, conforme descrito [aqui](https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/text/Tokenizer#arguments). Você pode tentar modificar esses argumentos na próxima célula abaixo e comparar o resultado com o gerado acima.

In [3]:
# Define input sentences
sentences = ["i love my dog", "I, love my cat", "You love my dog!"]

# Initialize the Tokenizer class
tokenizer = Tokenizer(num_words=1)

# Generate indices for each word in the corpus
tokenizer.fit_on_texts(sentences)

# Get the indices and print it
word_index = tokenizer.word_index
print(word_index)

{'love': 1, 'my': 2, 'i': 3, 'dog': 4, 'cat': 5, 'you': 6}


## Text to Sequences

No laboratório anterior, você viu como gerar um dicionário `word_index` para gerar tokens para cada palavra no seu corpus. Você pode então usar o resultado para converter cada uma das frases de entrada em uma sequência de tokens. Isso é feito utilizando o método [`texts_to_sequences()`](https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/text/Tokenizer#texts_to_sequences) conforme mostrado abaixo.

In [4]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Define your input texts
sentences = [
    'I love my dog',
    'I love my cat',
    'You love my dog!',
    'Do you think my dog is amazing?'
]

# Initialize the Tokenizer class
tokenizer = Tokenizer(num_words = 100, oov_token="<OOV>")

# Tokenize the input sentences
tokenizer.fit_on_texts(sentences)

# Get the word index dictionary
word_index = tokenizer.word_index

# Generate list of token sequences
sequences = tokenizer.texts_to_sequences(sentences)

# Print the result
print("\nWord Index = " , word_index)
print("\nSequences = " , sequences)


Word Index =  {'<OOV>': 1, 'my': 2, 'love': 3, 'dog': 4, 'i': 5, 'you': 6, 'cat': 7, 'do': 8, 'think': 9, 'is': 10, 'amazing': 11}

Sequences =  [[5, 3, 2, 4], [5, 3, 2, 7], [6, 3, 2, 4], [8, 6, 9, 2, 4, 10, 11]]


## Padding

Conforme mencionado no vídeo, você geralmente precisará preencher (pad) as sequências para um comprimento uniforme, pois é isso que o seu modelo espera. Você pode usar o método [pad_sequences](https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/sequence/pad_sequences) para isso. Por padrão, ele preencherá de acordo com o comprimento da sequência mais longa. Você pode substituir isso com o argumento `maxlen` para definir um comprimento específico. Sinta-se à vontade para experimentar [outros argumentos](https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/sequence/pad_sequences#args) mostrados em aula e comparar o resultado.

In [5]:
# Pad the sequences to a uniform length
padded = pad_sequences(sequences, maxlen=5)

# Print the result
print("\nPadded Sequences:")
print(padded)


Padded Sequences:
[[ 0  5  3  2  4]
 [ 0  5  3  2  7]
 [ 0  6  3  2  4]
 [ 9  2  4 10 11]]


## Tokens fora do vocabulário

Observe que você definiu um `oov_token` quando o `Tokenizer` foi inicializado anteriormente. Isso será usado quando você tiver palavras de entrada que não são encontradas no dicionário `word_index`. Por exemplo, você pode decidir coletar mais texto após o treinamento inicial e optar por não regenerar o `word_index`. Você verá isso em ação na célula abaixo. Note que o token `1` é inserido para palavras que não são encontradas no dicionário.

In [6]:
# Try with words that the tokenizer wasn't fit to
test_data = [
    'i really love my dog',
    'my dog loves my manatee'
]

# Generate the sequences
test_seq = tokenizer.texts_to_sequences(test_data)

# Print the word index dictionary
print("\nWord Index = " , word_index)

# Print the sequences with OOV
print("\nTest Sequence = ", test_seq)

# Print the padded result
padded = pad_sequences(test_seq, maxlen=10)
print("\nPadded Test Sequence: ")
print(padded)


Word Index =  {'<OOV>': 1, 'my': 2, 'love': 3, 'dog': 4, 'i': 5, 'you': 6, 'cat': 7, 'do': 8, 'think': 9, 'is': 10, 'amazing': 11}

Test Sequence =  [[5, 1, 3, 2, 4], [2, 4, 1, 2, 1]]

Padded Test Sequence: 
[[0 0 0 0 0 5 1 3 2 4]
 [0 0 0 0 0 2 4 1 2 1]]


# Ungraded Lab: Tokenizing the Sarcasm Dataset

Neste laboratório, você aplicará o que aprendeu nos dois exercícios anteriores para pré-processar o [News Headlines Dataset for Sarcasm Detection](https://www.kaggle.com/rmisra/news-headlines-dataset-for-sarcasm-detection/home). Este dataset contém manchetes de notícias que são rotuladas como sarcásticas ou não. Você revisitará este dataset em laboratórios posteriores, por isso é bom se familiarizar com ele agora.

## Baixar e inspecionar o dataset

Primeiro, você irá buscar o dataset e visualizar alguns de seus elementos.

In [10]:
# Download the dataset
!wget https://storage.googleapis.com/tensorflow-1-public/course3/sarcasm.json

--2024-06-30 02:35:19--  https://storage.googleapis.com/tensorflow-1-public/course3/sarcasm.json
Resolving storage.googleapis.com (storage.googleapis.com)... 142.250.219.155, 142.251.132.59, 172.217.30.59, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.250.219.155|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5643545 (5.4M) [application/json]
Saving to: ‘sarcasm.json’

sarcasm.json        100%[===================>]   5.38M  3.43MB/s    in 1.6s    

2024-06-30 02:35:21 (3.43 MB/s) - ‘sarcasm.json’ saved [5643545/5643545]



O dataset está salvo como um arquivo [JSON](https://www.json.org/json-en.html) e você pode usar o módulo [`json`](https://docs.python.org/3/library/json.html) do Python para carregá-lo em seu ambiente de trabalho. A célula abaixo desempacota o arquivo JSON em uma lista.

In [12]:
import json

# Load the JSON file
with open("./sarcasm.json", 'r') as f:
    datastore = json.load(f)

Você pode inspecionar alguns dos elementos na lista. Você notará que cada elemento consiste em um dicionário com um link de URL, a manchete real e um rótulo chamado `is_sarcastic`. Abaixo estão impressos dois elementos com rótulos contrastantes.

In [13]:
# Non-sarcastic headline
print(datastore[0])

# Sarcastic headline
print(datastore[20000])

{'article_link': 'https://www.huffingtonpost.com/entry/versace-black-code_us_5861fbefe4b0de3a08f600d5', 'headline': "former versace store clerk sues over secret 'black code' for minority shoppers", 'is_sarcastic': 0}
{'article_link': 'https://www.theonion.com/pediatricians-announce-2011-newborns-are-ugliest-babies-1819572977', 'headline': 'pediatricians announce 2011 newborns are ugliest babies in 30 years', 'is_sarcastic': 1}


Com isso, você pode coletar todas as URLs, manchetes e rótulos para facilitar o processamento ao usar o tokenizer. Para este laboratório, você precisará apenas das manchetes, mas incluímos o código para coletar as URLs e os rótulos também.

In [14]:
# Initialize lists
sentences = [] 
labels = []
urls = []

# Append elements in the dictionaries into each list
for item in datastore:
    sentences.append(item['headline'])
    labels.append(item['is_sarcastic'])
    urls.append(item['article_link'])

## Pré-processamento das manchetes

Você pode converter a lista de `sentences` acima em sequências preenchidas (padded) usando os mesmos métodos que você vem utilizando nos exercícios anteriores. A célula abaixo gera o dicionário `word_index` e gera a lista de sequências preenchidas para cada uma das 26.709 manchetes.

In [15]:
sentences[:3]

["former versace store clerk sues over secret 'black code' for minority shoppers",
 "the 'roseanne' revival catches up to our thorny political mood, for better and worse",
 "mom starting to fear son's web series closest thing she will have to grandchild"]

In [17]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Initialize the Tokenizer class
tokenizer = Tokenizer(oov_token="<OOV>")

# Generate the word index dictionary
tokenizer.fit_on_texts(sentences)

# Print the length of the word index
word_index = tokenizer.word_index
print(f'number of words in word_index: {len(word_index)}')

# Print the word index
# print(f'word_index: {word_index}')
print()

# Generate and pad the sequences
sequences = tokenizer.texts_to_sequences(sentences)
padded = pad_sequences(sequences, padding='post')

# Print a sample headline
index = 2
print(f'sample headline: {sentences[index]}')
print(f'padded sequence: {padded[index]}')
print()

# Print dimensions of padded sequences
print(f'shape of padded sequences: {padded.shape}')

number of words in word_index: 29657

sample headline: mom starting to fear son's web series closest thing she will have to grandchild
padded sequence: [  145   838     2   907  1749  2093   582  4719   221   143    39    46
     2 10736     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0]

shape of padded sequences: (26709, 40)
